In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from matplotlib.ticker import FuncFormatter
from preprocessing import *
import scipy.optimize as op
plt.style.use("dark_background") # Config plots for dark mode, delete if on light mode
plt.rcParams['figure.dpi'] = 150 # Hi-res plots

In [39]:
def Nprime(station_id, t):
    filter_end_stations = bike_data[bike_data.end_id == station_id]
    return len(filter_end_stations[filter_end_stations.end_time < t])

def N(station_id, t):
    filter_start_stations = bike_data[bike_data.start_id == station_id]
    return len(filter_start_stations[filter_start_stations.start_time < t])

def Nmax(station_id):
    return bike_data[bike_data.start_id == station_id]

def compensator_m3(station_id, t, alpha, beta, lambda_p):
    Nprime_comp = Nprime(station_id, t)
    summation = (alpha/beta)*np.sum(np.exp(-beta*(t-end_times_per_station_sorted[station_id][:Nprime_comp]))-1)
    return lambda_p*t - summation

def genB(station_id, beta):
    B = [0]
    ts = start_times_per_station_sorted[station_id]
    tprimes = end_times_per_station_sorted[station_id]
    for h in range(1, Nmax(station_id)):
        Nprimeh = Nprime(station_id, ts[h])
        Nprimeh_minus_one = Nprime(station_id, ts[h-1])
        tprimes_trunc = tprimes[Nprimeh_minus_one:Nprimeh]
        B_h = np.exp(-beta*(ts[h] - ts[h-1]))*B[h-1] + np.sum(np.exp(-beta*(ts[h] - tprimes_trunc)))
        B.append(B_h)
    return np.array(B)
genB(37, 1)

def log_likelihood(station_id, alpha,beta,lambda_p ):
    T = start_times_per_station_sorted[station_id][-1]
    B = genB(station_id, beta)
    trunc_B = B[:Nmax(station_id)]
    l_i = np.log(lambda_p + alpha*trunc_B)
    return l_i - compensator_m3(station_id, T, alpha, beta, lambda_p)
        
